## Working with data

In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [14]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
  root ="data",
  train = True,
  download = True,
  transform = ToTensor(),
)

# Download test data from open datasets,
test_data = datasets.FashionMNIST(
  root="data",
  train = False,
  download = True,
  transform=ToTensor(),
)

In [20]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"shape of X [N, C, H, W]: {X.shape}")
  print(f"shape of y: {y.shape} {y.type}")
  break


shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) <built-in method type of Tensor object at 0x168d37ed0>


## Creating Models

In [21]:
# Get cpu, gpu or mps device for training.
device = (
  "cuda"
  if torch.cuda.is_available()
  else "mps"
  if torch.backends.mps.is_available()
  else "cpu"

)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
  def __init__(self, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512,10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = NeuralNetwork().to(device)
print(model)


  
  


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309913  [   64/60000]
loss: 2.292226  [ 6464/60000]
loss: 2.278816  [12864/60000]
loss: 2.275843  [19264/60000]
loss: 2.253360  [25664/60000]
loss: 2.236523  [32064/60000]
loss: 2.234826  [38464/60000]
loss: 2.205237  [44864/60000]
loss: 2.205056  [51264/60000]
loss: 2.173664  [57664/60000]
Test Error: 
 Accuracy: 49.0%, Avg loss: 2.172435 

Epoch 2
-------------------------------
loss: 2.185074  [   64/60000]
loss: 2.173105  [ 6464/60000]
loss: 2.126087  [12864/60000]
loss: 2.139065  [19264/60000]
loss: 2.084469  [25664/60000]
loss: 2.038481  [32064/60000]
loss: 2.057894  [38464/60000]
loss: 1.985729  [44864/60000]
loss: 1.992570  [51264/60000]
loss: 1.921357  [57664/60000]
Test Error: 
 Accuracy: 56.8%, Avg loss: 1.925731 

Epoch 3
-------------------------------
loss: 1.960535  [   64/60000]
loss: 1.929610  [ 6464/60000]
loss: 1.828491  [12864/60000]
loss: 1.861890  [19264/60000]
loss: 1.747952  [25664/60000]
loss: 1.712430  [32064/600

## Saving Models

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

In [27]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [28]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')




Predicted: "Ankle boot", Actual: "Ankle boot"
